## 1. Load and preprocess data

In [2]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

DATA_PATH = "Data"
NUM_FEATURES = 1662
WINDOW_SIZE = 100  # sliding window size
STEP_SIZE = 10     # sliding window step

# Take only the first 10 words
all_labels = sorted(os.listdir(DATA_PATH))[:10]
label_map = {label: i for i, label in enumerate(all_labels)}

X, y = [], []

def normalize_keypoints(seq):
    """Normalize keypoints relative to first pose point"""
    seq = np.array(seq)
    if seq.shape[0] == 0:
        return seq
    ref = seq[0, :4]  # x,y,z,visibility of first pose landmark
    seq[:, :4] -= ref
    return seq

for label in all_labels:
    folder = os.path.join(DATA_PATH, label)
    for file in os.listdir(folder):
        if file.endswith(".npy"):
            seq = np.load(os.path.join(folder, file))
            seq = normalize_keypoints(seq)
            
            # Create sliding windows
            for start in range(0, len(seq) - WINDOW_SIZE + 1, STEP_SIZE):
                X.append(seq[start:start+WINDOW_SIZE])
                y.append(label_map[label])

X = np.array(X)
y = to_categorical(y).astype(int)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

X_train: (21, 100, 1662)
y_train: (21, 10)


## 2. Define and compile the model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, Bidirectional, Input

model = Sequential([
    Input(shape=(WINDOW_SIZE, NUM_FEATURES)),
    Masking(mask_value=0.0),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(128)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(len(all_labels), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 100, 1662)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 256)       │     1,833,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 256)            │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,245,322 (8.57 MB)

 Trainable params: 2,245,322 (8.57 MB)

 Non-trainable params: 0 (0.00 B)

## 3. Train the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=16,
    callbacks=[early_stop]
)

loss, acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {acc * 100:.4f}%")

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.0000e+00 - loss: 2.2499 - val_accuracy: 0.3333 - val_loss: 1.8267
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 453ms/step - accuracy: 0.2946 - loss: 1.8867 - val_accuracy: 0.3333 - val_loss: 1.7778
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 465ms/step - accuracy: 0.2946 - loss: 1.8988 - val_accuracy: 0.3333 - val_loss: 1.7443
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 478ms/step - accuracy: 0.3056 - loss: 1.8769 - val_accuracy: 0.3333 - val_loss: 1.7760
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 471ms/step - accuracy: 0.3681 - loss: 1.6791 - val_accuracy: 0.1667 - val_loss: 1.7989
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 473ms/step - accuracy: 0.1687 - loss: 1.9473 - val_accuracy: 0.1667 - val_loss: 1.8037
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 474ms/step - accuracy: 0.3790 - loss: 1.7235 - val_accuracy: 0.1667 - val_loss: 1.8260
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 463ms/step - accuracy: 0.2946 - loss: 1.6990 - val_accuracy: 0.1667 - val_los